In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="6P67MF6RktnoSQ9uM0JjqTlnW",
    project_name="tag-recommendation",
    workspace="vieiraeduardos",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/vieiraeduardos/tag-recommendation/f1f56fd3897e48f29a7e14c5a2698a0d



## Utils

Funções auxiliares

In [2]:
import numpy as np
from sklearn import preprocessing

def tags_list_to_array(tags_set):
  results = []
  for tags in tags_set:
    results.append(list(filter(None, tags.split(";"))))

  return results

def get_unique_tags_list(tags_set):

  results = []
  for tags in tags_set:
    for tag in tags:
      results.append(tag)

  return list(set(results))

def get_all_tags(tags_set):
  results = []
  for tags in tags_set:
    for tag in tags:
      results.append(tag)

  return results

def transform_array_to_binary(tags_set):
  le = preprocessing.LabelEncoder()
  uniques = get_unique_tags_list(dt.Tags)

  le.fit(uniques)
  
  length = len(uniques)
  results = []
  for tags in tags_set:
    encoder_result = le.transform(tags)

    arr = np.zeros((length,), dtype=int)
    for position in encoder_result:
      arr[position] = 1

    results.append(list(arr))
  return np.array(results)

Métrica de Hit ratio

In [3]:
import numpy as np

def hitk_convert_array_to_dict(arr):
  results = []

  for i in range(len(arr)):
    results.append(i)
    results.append(arr[i])

  return dict(zip(results[::2], results[1::2]))

def hitk_sort_dict(x):
  return sorted(x.items(), key=lambda kv: kv[1], reverse=True)

def hitk_get_k_keys(x, k):
  keys = []

  for item in x[:k]:
    key = item[0]
    value = item[1]

    if value > 0.5:
      keys.append(item[0])

  return keys

def hitk_get_number_of_correct_keys(keys_from_y_true, keys_from_y_scores):
  count = 0

  for i in range(min(len(keys_from_y_true), len(keys_from_y_scores))):
    if keys_from_y_scores[i] in keys_from_y_true:
      count += 1

  return count

def hitk_calculate(y_true, y_scores, k = 1):
  # converting from array format to dict format
  y_scores = hitk_convert_array_to_dict(y_scores)
  y_true = hitk_convert_array_to_dict(y_true)

  # sorting dict by value
  y_scores = hitk_sort_dict(y_scores)
  y_true = hitk_sort_dict(y_true)

  # getting k first keys from dict
  keys_from_y_scores = hitk_get_k_keys(y_scores, k)
  keys_from_y_true = hitk_get_k_keys(y_true, len(y_true)) 

  # process
  m = hitk_get_number_of_correct_keys(keys_from_y_true, keys_from_y_scores)

  # number of tags truely
  ng = len(keys_from_y_true)

  # number of tags recommend
  nr = len(keys_from_y_scores)

  if min(ng,nr) == 0:
    if ng == nr:
        return 1
    else:
        return 0
  else:
    return m / min(ng, nr)

def hitk(y_true, y_scores, k = 1):
  results = []
  
  for i in range(len(y_true)):
    value = hitk_calculate(y_true[i], y_scores[i], k = k)
  
    results.append(value)

  return np.mean(results)


In [4]:
from sklearn.metrics import average_precision_score

def ap(y_true, y_scores):
  results = []
  
  for i in range(len(y_true)):
    value = average_precision_score(y_true[i], y_scores[i])
  
    results.append(value)

  return np.mean(results)

## Pré processamento

Carregando bibliotecas

In [5]:
import pandas as pd
import numpy as np

Carregando base de dados

In [6]:
dt = pd.read_csv("Datasets/Blogs.csv")

print('Total Rows: {}\nTotal Columns: {}'.format(dt.shape[0], dt.shape[1]))
print('Columns:', dt.columns.to_list())

Total Rows: 9397
Total Columns: 9
Columns: ['AdjustedTitle', 'ImagePath', 'Title', 'URL', 'Description', 'Tags', 'MicroName', 'Category', 'Content']


In [7]:
dt.head()

,AdjustedTitle,ImagePath,Title,URL,Description,Tags,MicroName,Category,Content
0,free diet plan from julia,Images/Free Diet Plan from Julia/1.jpg,Free Diet Plan from Julia,http://juliahavey.typepad.com,"Opinions on diet plans, weight loss, nutrition...",effect;weighting;egress;publication;supply;off...,weight issues,Health,Free Diet Plan from Julia11 de jul. de 2011 — ...
1,retro gaming,Images/Retro Gaming/5.jpg,Retro Gaming,http://www.racketboy.com/,Notes and thoughts on video gaming with the ol...,sega;comfort;cabinet;console platforms;solace;...,console platforms,Entertainment,"Top 90 Retro Gaming Blogs, Websites & Influenc..."
2,fancy pigeons india,Images/Fancy Pigeons India/5.jpg,Fancy Pigeons India,http://www.fancypigeonsindia.blogspot.com/,"Offers posts on exhibitions, sales, rare breed...",deary;parrots;ducky;skirt;razzing;snort;favori...,birds,Pets,Fancy Pigeons India
3,bookish gardener,Images/Bookish Gardener/0.jpg,Bookish Gardener,http://fortyfour.typepad.com/bookishgardener/,Reviews of gardening books and general comment...,hobbies and lifestyle;falco subbuteo;life styl...,gardening,Hobbies and Lifestyle,Bookish Gardener9 de jan. de 2020 — occasional...
4,goatworld,Images/GoatWorld/1.jpg,GoatWorld,http://goatworld.com/,Presents current goat news topics and archives...,deary;ducky;stock;favorite;capricorn;united ki...,livestock goats,Pets,Welcome to GoatWorld - GOATWORLD.COMEducationa...


In [9]:
X = dt[['AdjustedTitle', 'ImagePath', 'Title', 'URL', 'Description', 'MicroName', 'Category', 'Content', 'Tags']]

X.shape

(9397, 9)

Convertendo formato de tags de STRING para ARRAY

In [10]:
dt.Tags = tags_list_to_array(dt.Tags)

dt.shape

(9397, 9)

Convertendo formato de tags de ARRAY de string para ARRAY binário

Splitting train and test sets

In [10]:
from skmultilearn.model_selection import iterative_train_test_split

def iterative_train_test_split_dataframe(X, y, test_size):
    df_index = np.expand_dims(X.index.to_numpy(), axis=1)
    X_train, y_train, X_test, y_test = iterative_train_test_split(df_index, y, test_size = test_size)
    X_train = X.loc[X_train[:,0]]
    X_test = X.loc[X_test[:,0]]
    return X_train, y_train, X_test, y_test

X_test, y_test, X_train, y_train = iterative_train_test_split_dataframe(X, y, test_size=0.9)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7504, 9), (1893, 9), (7504, 3861), (1893, 3861))

In [12]:
X_train.to_csv("Datasets/Train.csv", index=0)
X_test.to_csv("Datasets/Test.csv", index=0)

### Loading

In [11]:
train = pd.read_csv("Datasets/Train.csv")
test = pd.read_csv("Datasets/Test.csv")

In [12]:
train.Tags = tags_list_to_array(train.Tags)

train.shape

(7504, 9)

In [13]:
le = preprocessing.LabelEncoder()
uniques = get_unique_tags_list(dt.Tags)

le.fit(uniques)
  
length = len(uniques)
results = []

for tags in train.Tags:
    encoder_result = le.transform(tags)

    arr = np.zeros((length,), dtype=int)

    for position in encoder_result:
      arr[position] = 1

    results.append(list(arr))

y_train = np.array(results)

y_train.shape

(7504, 3861)

In [15]:
test.Tags = tags_list_to_array(test.Tags)

test.shape

(1893, 9)

In [16]:
results = []

for tags in test.Tags:
    encoder_result = le.transform(tags)

    arr = np.zeros((length,), dtype=int)

    for position in encoder_result:
      arr[position] = 1

    results.append(list(arr))

y_test = np.array(results)

y_test.shape

(1893, 3861)

## Modelo

In [21]:
import cv2
from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

Extraindo embeddings das entradas 

In [30]:
def get_image_features(images_paths, embed):
  images = []

  for filename in tqdm(images_paths):
    img = cv2.imread("Datasets/" + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224)) 
    img = np.array(img, dtype=np.uint8)

    images.append(img)
   
  result = embed([images[0]])

  for image in tqdm(images[1:]):
    embeddings_image = embed([image])
    result = tf.concat([result, embeddings_image], 0)
      
  return result.numpy()

In [23]:
def get_text_features(texts, embed):
  embeddings = embed(tf.constant([texts[0]]))

  for text in tqdm(texts[1:]):
    text = tf.constant([text])
    tmp = embed(text)
    embeddings = tf.concat([embeddings, tmp], 0)
      
  return embeddings.numpy()

In [24]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed = hub.KerasLayer(module_url, trainable=False, name='USE_Embedding')

In [25]:
module_url = "https://tfhub.dev/google/imagenet/inception_v1/feature_vector/5"
INCEPTION_embed = hub.KerasLayer(module_url, trainable=False, name="Inception_Embedding")

Generating embeddings for train set

In [26]:
titles_embeddings = X_train.Title.apply(str).to_list()
titles_embeddings = get_text_features(titles_embeddings, embed)

100%|██████████| 7503/7503 [00:48<00:00, 155.93it/s]


In [27]:
descriptions_embeddings = X_train.Description.apply(str).to_list()
descriptions_embeddings = get_text_features(descriptions_embeddings, embed)

100%|██████████| 7503/7503 [00:45<00:00, 165.14it/s]


In [28]:
contents_embeddings = X_train.Content.apply(str).to_list()
contents_embeddings = get_text_features(contents_embeddings, embed)

100%|██████████| 7503/7503 [00:46<00:00, 162.83it/s]


In [31]:
images_embeddings = X_train['ImagePath'].apply(str).to_numpy().tolist()
images_embeddings = get_image_features(images_embeddings, INCEPTION_embed)

100%|██████████| 7503/7503 [00:57<00:00, 131.18it/s]


Saving embedings for train set

In [32]:
np.save("Embeddings/titles_train_x.npy", titles_embeddings)
np.save("Embeddings/descriptions_train_x.npy", descriptions_embeddings)
np.save("Embeddings/contents_train_x.npy", contents_embeddings)
np.save("Embeddings/images_train_x.npy", images_embeddings)

Generating embeddings for test set

In [38]:
titles_embeddings = X_test.Title.apply(str).to_list()
titles_embeddings = get_text_features(titles_embeddings, embed)

100%|██████████| 1892/1892 [00:11<00:00, 157.95it/s]


In [39]:
descriptions_embeddings = X_test.Description.apply(str).to_list()
descriptions_embeddings = get_text_features(descriptions_embeddings, embed)

100%|██████████| 1892/1892 [00:11<00:00, 166.36it/s]


In [40]:
contents_embeddings = X_test.Content.apply(str).to_list()
contents_embeddings = get_text_features(contents_embeddings, embed)

100%|██████████| 1892/1892 [00:11<00:00, 162.68it/s]


In [41]:
images_embeddings = X_test['ImagePath'].apply(str).to_numpy().tolist()
images_embeddings = get_image_features(images_embeddings, INCEPTION_embed)

100%|██████████| 1892/1892 [00:13<00:00, 140.33it/s]


Saving embeddings for test set

In [42]:
np.save("Embeddings/titles_test_x.npy", titles_embeddings)
np.save("Embeddings/descriptions_test_x.npy", descriptions_embeddings)
np.save("Embeddings/contents_test_x.npy", contents_embeddings)
np.save("Embeddings/images_test_x.npy", images_embeddings)

Loading embeddings for train and test sets

In [17]:
titles_train_x = np.load("Embeddings/titles_train_x.npy")
descriptions_train_x = np.load("Embeddings/descriptions_train_x.npy")
contents_train_x = np.load("Embeddings/contents_train_x.npy")
images_train_x = np.load("Embeddings/images_train_x.npy")

titles_test_x = np.load("Embeddings/titles_test_x.npy")
descriptions_test_x = np.load("Embeddings/descriptions_test_x.npy")
contents_test_x = np.load("Embeddings/contents_test_x.npy")
images_test_x = np.load("Embeddings/images_test_x.npy")

In [18]:
from keras.models import Sequential
from keras.layers import Dense
import pickle 
#!pip install iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [19]:
from sklearn.metrics import coverage_error
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.metrics import label_ranking_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import hamming_loss

Training

In [30]:
def get_model(n_inputs, n_outputs):
  model = Sequential()
  model.add(Dense(n_inputs * 2, activation='relu'))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam')
  return model

def evaluate_model(X, y, variable):
  n_inputs, n_outputs = X.shape[1], len(get_unique_tags_list(dt.Tags))

  mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=0)
  
  fold_no = 1
  best_result = 0
    
  for train_ix, test_ix in mskf.split(X, y):
    print(f'Score for fold {fold_no}')
    
    train_ix = np.array(train_ix)
    test_ix = np.array(test_ix)

    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    
    model = get_model(n_inputs, n_outputs)
    
    model.fit(X_train, y_train, verbose=0, epochs=50)
    
    predictions = model.predict(X_test)

    print("\nMultilabel ranking metrics")
    h_score = hitk(y_test, predictions, 5)
    ap_score = label_ranking_average_precision_score(y_test, predictions)
    
    print("Coverage Error:", coverage_error(y_test, predictions))
    print("Label Ranking Average Precision:", label_ranking_average_precision_score(y_test, predictions))
    print("Label Ranking Loss:", label_ranking_loss(y_test, predictions))
    print("Hit Ratio:", h_score)
    print("Average Precision:", ap_score)

    predictions = predictions.round()
    
    hamming_loss_score = hamming_loss(y_test, predictions)
    print("Hamming Loss:", hamming_loss_score)

    print("\nNormal ranking metrics")
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')
    
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    
    if best_result < h_score:
        bst_result = h_score
        
        model.save_weights("/Models/MLP")
        
    metrics = {
        "f1_train_mlp_" + variable: f1,
        "recall_train_mlp_" + variable: recall,
        "precision_train_mlp_" + variable: precision,
        "hit_ratio_5_train_mlp_" + variable: h_score,
        "ap_train_mlp_"+ variable: ap_score,
        "hamming_loss_train_mlp_" + variable: hamming_loss_score
    }
    
    params = {
        "random_state": "",
        "model_type":"logreg",
        "scaler":"",
        "param_grid": "",
        "stratify": True
    }
    
    experiment.log_parameters(params, step=fold_no)
    experiment.log_metrics(metrics, step=fold_no)
    
    fold_no += 1
  
  return best_result

In [31]:
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn

In [32]:
variables = {
    "title": titles_train_x,
    "description": descriptions_train_x,
    "content": contents_train_x,
    "image": images_train_x
}

for variable in variables.keys(): 
  print("Analysing variable", variable)

  # load dataset
  X = variables[variable]

  # evaluate model
  evaluate_model(X, y_train, variable)

Analysing variable title
Score for fold 1

Multilabel ranking metrics
Coverage Error: 851.7690253671562
Label Ranking Average Precision: 0.38146230734156
Label Ranking Loss: 0.11212294990374125
Hit Ratio: 0.4056964842011571
Average Precision: 0.38146230734156
Hamming Loss: 0.0049618087001264575

Normal ranking metrics
Precision: 0.41993766390724896
Recall: 0.277190332326284
F1 Score: 0.32212013147608354
Score for fold 2

Multilabel ranking metrics
Coverage Error: 836.1597869507324
Label Ranking Average Precision: 0.3996431714109097
Label Ranking Loss: 0.10495930996499629
Hit Ratio: 0.4324456280514869
Average Precision: 0.3996431714109097
Hamming Loss: 0.0049613551610888495

Normal ranking metrics
Precision: 0.4190075049691754
Recall: 0.2924617038430529
F1 Score: 0.3331879166686888
Score for fold 3

Multilabel ranking metrics
Coverage Error: 842.9853333333333
Label Ranking Average Precision: 0.38591534217417733
Label Ranking Loss: 0.10620047052656942
Hit Ratio: 0.4328666666666666
Averag

Coverage Error: 719.9347536617843
Label Ranking Average Precision: 0.4100515865588897
Label Ranking Loss: 0.08397386835190081
Hit Ratio: 0.4470483799378607
Average Precision: 0.4100515865588897
Hamming Loss: 0.00486548023165866

Normal ranking metrics
Precision: 0.44512889029724334
Recall: 0.28003224939532384
F1 Score: 0.3289085162574284
Score for fold 3

Multilabel ranking metrics
Coverage Error: 764.1493333333333
Label Ranking Average Precision: 0.42051900082405524
Label Ranking Loss: 0.09063888589071338
Hit Ratio: 0.4630222222222222
Average Precision: 0.42051900082405524
Hamming Loss: 0.005002158335491669

Normal ranking metrics
Precision: 0.4431282147273626
Recall: 0.2704518508760374
F1 Score: 0.3235155372301045
Score for fold 4

Multilabel ranking metrics
Coverage Error: 759.4986631016043
Label Ranking Average Precision: 0.4254887845907349
Label Ranking Loss: 0.08923276392929518
Hit Ratio: 0.4779188948306596
Average Precision: 0.4254887845907349
Hamming Loss: 0.004870451394515566


In [33]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/vieiraeduardos/tag-recommendation/f1f56fd3897e48f29a7e14c5a2698a0d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     ap_train_mlp_content [10]               : (0.383539847304143, 0.42991149886807245)
COMET INFO:     ap_train_mlp_description [10]           : (0.5663640275138816, 0.6108056664977085)
COMET INFO:     ap_train_mlp_image [10]                 : (0.11726767619788013, 0.1478940777140308)
COMET INFO:     ap_train_mlp_title [10]                 : (0.3727146338330894, 0.40612313318591886)
COMET INFO:     batch_loss [48400]                      : (0.0003269795561209321, 1.075533390045166)
COMET INFO:     epoch_duration [2200]                   : (0.34007376339286566, 1.1024644961580634)
COMET INFO:     f1_train_mlp_content [10]        